## Conclusions:
- Use this pattern to creat a routing agent that routes each customer requests to expert agents accordingly. 
- Use LangGraph ToolNode to host tools.
- Use LangChain @tool annotation to define the metadata of tools
- LangGraph ToolExecutor is able to augment the prompts with the tools metadata and extract the tools id into the state. 
- llm.bind_tools() from langchain is not neccessary when use ToolExecutor. 

In [1]:
# Create three LangGraph agents. Each agent has its own StateGraph workflow. 
# The three agents compile to separate graphs. 
# The router agent decides whether the technical support agent or the customer service agent is suitable to service each customer request. 
# The router agent then routs the customer request to the service agent accordingly for the continued process.

from typing import Annotated, Any, Dict, List, Tuple, TypedDict
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage
from langgraph.graph import StateGraph, END
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from crm_chatbot.tools.llm_choice import instantiate_chatllm

# Define separate state types for each agent
class RouterState(TypedDict):
    messages: List[BaseMessage]
    selected_agent: str

class TechnicalSupportState(TypedDict):
    messages: List[BaseMessage]
    issue_status: str  # e.g., 'open', 'in_progress', 'resolved'
    technical_metadata: Dict[str, Any]

class CustomerServiceState(TypedDict):
    messages: List[BaseMessage]
    ticket_status: str  # e.g., 'open', 'pending', 'closed'
    customer_metadata: Dict[str, Any]

# Router Agent Graph
def create_router_graph():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a router agent that determines which specialized assistant should handle a customer request.
        
        Available assistants:
        - technical_support: Handles technical issues, product functionality, and setup questions
        - customer_service: Handles billing, account management, and general inquiries
        
        Respond only with either 'technical_support' or 'customer_service'."""),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    model = instantiate_chatllm(model='qwen2.5:14b', temperature=0)
    chain = prompt | model
    
    def route_request(state: RouterState) -> Dict:
        messages = state["messages"]
        response = chain.invoke({"messages": messages})
        return {"selected_agent": response.content}
    
    # Create router workflow
    workflow = StateGraph(RouterState)
    workflow.add_node("route", route_request)
    workflow.add_edge("route", END)
    workflow.set_entry_point("route")
    
    return workflow.compile()

# Technical Support Agent Graph
def create_technical_support_graph():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a technical support specialist. Help users with technical issues, product functionality, and setup questions.
        Be detailed and precise in your explanations. Track the status of the issue."""),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    model = instantiate_chatllm(model='qwen2.5:14b', temperature=0)
    chain = prompt | model
    
    def assess_issue(state: TechnicalSupportState) -> Dict[str, Any]:
        messages = state["messages"]
        if not messages:
            return {
                "issue_status": "open",
                "technical_metadata": {"initial_assessment": "pending"}
            }
        return {
            "issue_status": "in_progress",
            "technical_metadata": {"initial_assessment": "completed"}
        }
    
    def provide_support(state: TechnicalSupportState) -> Dict:
        messages = state["messages"]
        response = chain.invoke({"messages": messages})
        return {
            "messages": messages + [AIMessage(content=response.content)],
            "issue_status": "in_progress"
        }
    
    # Create technical support workflow
    workflow = StateGraph(TechnicalSupportState)
    workflow.add_node("assess", assess_issue)
    workflow.add_node("support", provide_support)
    
    # Add edges with conditions
    workflow.add_conditional_edges(
        "assess",
        lambda x: "support" if x["issue_status"] == "in_progress" else END
    )
    workflow.add_edge("support", END)
    
    workflow.set_entry_point("assess")
    
    return workflow.compile()

# Customer Service Agent Graph
def create_customer_service_graph():
    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are a customer service representative. Help users with billing, account management, and general inquiries.
        Be professional and courteous. Track the status of customer tickets."""),
        MessagesPlaceholder(variable_name="messages"),
    ])
    
    model = instantiate_chatllm(model='qwen2.5:14b', temperature=0)
    chain = prompt | model
    
    def evaluate_request(state: CustomerServiceState) -> Dict[str, Any]:
        messages = state["messages"]
        if not messages:
            return {
                "ticket_status": "open",
                "customer_metadata": {"priority": "normal"}
            }
        return {
            "ticket_status": "pending",
            "customer_metadata": {"priority": "normal"}
        }
    
    def handle_request(state: CustomerServiceState) -> Dict:
        messages = state["messages"]
        response = chain.invoke({"messages": messages})
        return {
            "messages": messages + [AIMessage(content=response.content)],
            "ticket_status": "in_progress"
        }
    
    # Create customer service workflow
    workflow = StateGraph(CustomerServiceState)
    workflow.add_node("evaluate", evaluate_request)
    workflow.add_node("handle", handle_request)
    
    # Add edges with conditions
    workflow.add_conditional_edges(
        "evaluate",
        lambda x: "handle" if x["ticket_status"] == "pending" else END
    )
    workflow.add_edge("handle", END)
    
    workflow.set_entry_point("evaluate")
    
    return workflow.compile()

# Multi-Agent System Manager
class MultiAgentSystem:
    def __init__(self):
        self.router_graph = create_router_graph()
        self.tech_support_graph = create_technical_support_graph()
        self.customer_service_graph = create_customer_service_graph()
    
    def process_request(self, user_message: str) -> Dict[str, Any]:
        # Initialize router state
        router_state: RouterState = {
            "messages": [HumanMessage(content=user_message)],
            "selected_agent": ""
        }
        
        # Get routing decision
        router_result = self.router_graph.invoke(router_state)
        selected_agent = router_result["selected_agent"]
        
        # Initialize appropriate agent state and process request
        if selected_agent == "technical_support":
            tech_state: TechnicalSupportState = {
                "messages": [HumanMessage(content=user_message)],
                "issue_status": "open",
                "technical_metadata": {}
            }
            return self.tech_support_graph.invoke(tech_state)
            
        elif selected_agent == "customer_service":
            cs_state: CustomerServiceState = {
                "messages": [HumanMessage(content=user_message)],
                "ticket_status": "open",
                "customer_metadata": {}
            }
            return self.customer_service_graph.invoke(cs_state)
            
        else:
            raise ValueError(f"Unknown agent type: {selected_agent}")

# Example usage
def main():
    # Create the multi-agent system
    system = MultiAgentSystem()
    
    # Process a sample request
    request = "I can't log into my account. It keeps saying invalid password."
    result = system.process_request(request)
    
    # Print the result
    print("Request:", request)
    print("\nResponse:")
    for message in result["messages"]:
        if isinstance(message, HumanMessage):
            print(f"Human: {message.content}")
        elif isinstance(message, AIMessage):
            print(f"Assistant: {message.content}")
        print(f"Status: {result.get('ticket_status') or result.get('issue_status')}")

if __name__ == "__main__":
    main()

Request: I can't log into my account. It keeps saying invalid password.

Response:
Human: I can't log into my account. It keeps saying invalid password.
Status: in_progress
Assistant: I'm sorry to hear you're having trouble logging in. Let's try a few steps to resolve this issue:

1. **Check Your Password**: Make sure there are no typos or extra spaces. Also, ensure that your Caps Lock key is not on if your password includes uppercase letters.

2. **Reset Your Password**: If you remember the email address associated with your account, you can reset your password through our "Forgot Password" feature. Follow the instructions to receive a link to set up a new password.

3. **Verify Account Information**: Ensure that the username or email and password combination matches exactly what is stored in our system.

4. **Contact Support**: If you've tried these steps and still can't log in, please contact our support team directly. We'll need some information from you to verify your identity bef